# DataFrame查、改、增、 删
这一节主要介绍如何对Series和DataFrame对象进行查、改、增、删。

In [38]:
__auther__ = 'zhenhang.sun@gmail.com'

In [39]:
pwd

'/Users/liuning/project/spark-cap/pandas-cookbook'

In [40]:
import numpy as np
import pandas as pd

# 2. DataFrame

DataFrame由具有共同索引的Series按列排列构成（2D），是使用最多的对象。

## 2.1 预览

In [41]:
data = [[1, 2, 3], [4, 5, 6]]
index = ['a', 'b']
columns = ['A', 'B', 'C']
df = pd.DataFrame(data=data, index=index, columns=columns)
df

,A,B,C
a,1,2,3
b,4,5,6


In [49]:
df.index  # 行索引

Index(['a', 'b'], dtype='object')

In [50]:
df.columns  # 列索引，由Series的name构成

Index(['A', 'B', 'C'], dtype='object')

In [51]:
df.values

array([[1, 2, 3],
       [4, 5, 6]])

In [52]:
df.dtypes  # 这里的dtype带s，查看每列元素类型

A    int64
B    int64
C    int64
dtype: object

## 2.2. 创建

##### `pd.DataFrame(data=None, index=None, columns=None)`
函数有多个参数，对我们有用的主要是：`data`,`index`和`columns`三项

### 2.2.1 data无 行索引，无 列索引
- 如果 data 为 **ndarray(2D) or list of list(2D)**，那么其缺少 DataFrame 需要的行、列索引信息；
- 如果提供 index 或 columns 项，其必须和data的行 或 列长度相同；
- 如果不提供 index 或 columns 项，那么其将默认生成数值索引range(0, data.shape[0])) 或 range(0, data.shape[1])。

In [53]:
# data = [[1,2,3],
#        [4,5,6]]
data1 = np.array([[1, 2, 3], [4, 5, 6]])
index1 = ['a', 'b']
columns1 = ['A', 'B', 'C']
df = pd.DataFrame(data=data1, index=index1, columns=columns1)
df

,A,B,C
a,1,2,3
b,4,5,6


### 2.2.2 data无 行索引，有 列索引
 - 如果data为 **dict of (ndarray(1D) or list(1D))**，所有ndarray或list的长度必须相同。dict的key为DataFrame提供了需要的columns信息，缺失index；
 - 如果提供 index 项，必须和list的长度相同；
 - 如果不提供 index，那么其将默认生成数值索引range(0, data.shape[0]))；
 - 如果还额外提供了columns项，那么其将对当前构建的DataFrame进行 **列重索引**。

In [29]:
data2 = {'A': [1, 4], 'B': [2, 5], 'C': [3, 6]}
index2 = ['a', 'b']
columns2 = ['A', 'B', 'D']
df = pd.DataFrame(data=data2, index=index2, columns=columns2)
df

,A,B,D
a,1,2,NaN
b,4,5,NaN


### 2.2.3 data有 行索引，有 列索引
 - 如果data为 **dict of (Series or dict)**，那么其已经提供了DataFrame需要的所有信息；
 - 如果多个Series或dict间的索引不一致，那么取并操作（pandas不会试图丢掉信息），缺失的数据填充NaN；
 - 如果提供了index项或columns项，那么其将对当前构建的DataFrame进行 重索引（reindex，pandas内部调用接口）。

In [56]:
# data3 = { 'A' : pd.Series([1,4] ,index = ['a','b']), 'B' : pd.Series([2,5] ,index = ['a','b']), 'C' : pd.Series([3,6] ,index = ['a','c']) }
data3 = {'A': {'a': 1, 'b': 4}, 'B': {'a': 2, 'b': 5}, 'C': {'a': 3, 'c': 6}}
df = pd.DataFrame(data=data3)
df

,A,B,C
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,NaN,NaN,6.0


## 2.1 查

### 2.1.1 []，快捷查看
**[]  属于快捷查看方式**，只包含下面四种，两种列操作、两种行操作。

In [58]:
# 列操作，单列索引，返回Series
df['A']

a    1.0
b    4.0
c    NaN
Name: A, dtype: float64

In [59]:
# 列操作，列索引列表，返回DataFrame
df[['A', 'C']]

,A,C
a,1.0,3.0
b,4.0,NaN
c,NaN,6.0


In [63]:
# df[0]  # 报错，0不是列名

In [64]:
df[0:1]  # 行操作，位置范围，返回DataFrame

,A,B,C
a,1.0,2.0,3.0


In [65]:
# df[[0,1]] #报错，不能这样使用

In [66]:
mask = [False, True, True]
df[mask]  # 行操作，mask，必须和行长度一致，返回DataFrame

,A,B,C
b,4.0,5.0,NaN
c,NaN,NaN,6.0


### 2.1.2 .loc[]，基于索引
.loc[]在DataFrame中与[]不一致。
- DataFrame 有两维，每一维都和 Series 的 .loc[] 用法相同；
- Series有四种方式，所以DataFrame有**16种**方式;
- 可以缺省后面维度，默认补全为 ':' 。

**下面都以第一维度为例，第二维可以类比。**

In [67]:
df

,A,B,C
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,NaN,NaN,6.0


In [68]:
df.loc['b', 'B']  # 返回单一值，因为两维都是单索引

5.0

In [69]:
df.loc['a':'b', 'B']  #返回Series，如果只有一维是单索引

a    2.0
b    5.0
Name: B, dtype: float64

In [72]:
df.loc[['a', 'b'], 'B']  #返回Series，如果只有一维是单索引

a    2.0
b    5.0
Name: B, dtype: float64

In [73]:
mask1 = [False, True, False]
df.loc[mask1, 'B']

b    5.0
Name: B, dtype: float64

### 2.1.3 .iloc[]，基于位置
无视索引，只按照位置定位。
- DataFrame 有两维，每一维都和 Series 的 .iloc[]用法相同；
- Series有四种方式，所以DataFrame有**16**种方式；
- 可以缺省后面维度，默认补全为 ':' 。

**下面都以第一维度为例，第二维可以类比。**

In [74]:
df

,A,B,C
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,NaN,NaN,6.0


In [75]:
df.iloc[1, 1]  # 返回单一值，因为两维都是scalar

5.0

In [76]:
df.iloc[0:2, 0]  # 返回Series，如果只有一维是scalar

a    1.0
b    4.0
Name: A, dtype: float64

In [77]:
df.iloc[[0, 1], [0, 2]]  # 返回DataFrame

,A,C
a,1.0,3.0
b,4.0,NaN


In [78]:
mask1 = [False, True, False]  # 返回DataFrame
mask2 = [True, False, True]
df.iloc[mask1, mask2]

,A,C
b,4.0,NaN


## 2.2 改

### 2.2.1 改值

**直接在2.1.2查的基础上赋值进行修改，.loc[]方法确保在原地修改。**

In [79]:
df

,A,B,C
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,NaN,NaN,6.0


In [80]:
#修改单值
df1 = df.copy()
df1.loc['a', 'A'] = 10
df1

,A,B,C
a,10.0,2.0,3.0
b,4.0,5.0,NaN
c,NaN,NaN,6.0


In [81]:
#修改单列
df1.loc[:, 'A'] = [100, 200, 300]
df1

,A,B,C
a,100,2.0,3.0
b,200,5.0,NaN
c,300,NaN,6.0


In [82]:
#修改多列
df1.loc[:, ['A', 'B']] = [[1, 2], [3, 4], [5, 6]]
df1

,A,B,C
a,1,2,3.0
b,3,4,NaN
c,5,6,6.0


In [83]:
#修改行，见下面的增减操作

##### 函数批量任意修改：`DataFrame.replace(to_replace=None, value=None, inplace=False)`
- to_replace：要修改的值，可以为列表;
- value：改为的值，可以为列表，与to_repalce要匹配；
- inplace：是否在原地修改；

In [84]:
df1

,A,B,C
a,1,2,3.0
b,3,4,NaN
c,5,6,6.0


In [85]:
df1.replace(to_replace=3, value=100, inplace=False)

,A,B,C
a,1,2,100.0
b,100,4,NaN
c,5,6,6.0


### 2.2.2 改索引

**直接在索引上改，索引类似于tuple，必须全改，不能切片修改。**

In [86]:
df1 = df.copy()
df1.index = ['e','f', 'g']
df1.columns = ['E','F','G']
df1

,E,F,G
e,1.0,2.0,3.0
f,4.0,5.0,NaN
g,NaN,NaN,6.0


##### 函数修改：`DataFrame.rename(index=None, column =None,  level=None, inplace=False)`
- index：list or dict，list类型时必须长度相同，dict类型时可以部分修改；
- columns：list or dict，list时必须长度相同，dict时可以部分修改；
- level：多重索引时，可以指定修改哪一重，目前还用不着；
- inplace：是否原地修改。

In [87]:
df1.rename(index={'e': 'b'}, columns={'E': 'A'}, inplace=False)

,A,F,G
b,1.0,2.0,3.0
f,4.0,5.0,NaN
g,NaN,NaN,6.0


## 2.3 增

### 2.3.1 直接增一行

In [88]:
df1 = df.copy()
df1.loc['c'] = [7, 8, 9]
df1

,A,B,C
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,7.0,8.0,9.0


### 2.3.2 函数增多行
##### `pd.concat(objs, axis=0)`
确保 **列索引** 相同，行增加。
（其实这个函数并不要求列索引相同，它可以选择出相同的列。而我写这个教程遵循了python的宣言—明确：做好一件事有一种最好的方法，精确控制每一步，可以少犯错。）
- objs: list of DataFrame；
- axis: 取0，行增加操作。

In [89]:
df1 = pd.DataFrame([[22, 33, 44], [55, 66, 77]],
                   index=['c', 'd'],
                   columns=['A', 'B', 'C'])
pd.concat([df, df1], axis=0)

,A,B,C
a,1.0,2.0,3.0
b,4.0,5.0,NaN
c,NaN,NaN,6.0
c,22.0,33.0,44.0
d,55.0,66.0,77.0


### 2.3.3 直接增一列

In [90]:
df1

,A,B,C
c,22,33,44
d,55,66,77


In [91]:
df1 = df.copy()
df1['H'] = [7, 8, 7]
df1

,A,B,C,H
a,1.0,2.0,3.0,7
b,4.0,5.0,NaN,8
c,NaN,NaN,6.0,7


### 2.3.4 函数增多列
##### `pd.concat(objs, axis=1)`
确保**行索引**相同，列增加
- objs: list of DataFrame；
- axis: 取1，列增加操作。

In [92]:
df1 = pd.DataFrame([[22, 33], [44, 55]], index=['a', 'b'], columns=['D', 'E'])
pd.concat([df, df1], axis=1)

,A,B,C,D,E
a,1.0,2.0,3.0,22.0,33.0
b,4.0,5.0,NaN,44.0,55.0
c,NaN,NaN,6.0,NaN,NaN


## 2.4 删

### 2.4.1 函数删多行
##### `DataFrame.drop(labels, axis=0,  level=None, inplace=False)`
- labels：索引，单索引或索引的列表；
- axis：0，删行；
- level：多重索引需要指定；
- inplace：是否本地修改。

In [93]:
df1 = df.copy()
df1.drop(['a'], axis=0)

,A,B,C
b,4.0,5.0,NaN
c,NaN,NaN,6.0


### 2.4.2 直接删一列

In [95]:
# 不建议使用
df1 = df.copy()
del df1['A']
df1

,B,C
a,2.0,3.0
b,5.0,NaN
c,NaN,6.0


### 2.4.3 函数删多列
##### `DataFrame.drop(labels, axis=1,  level=None, inplace=False)`
- labels：索引，单索引或索引的列表；
- axis：1，删列；
- level：多重索引需要指定；
- inplace：是否本地修改。

In [96]:
df1 = df.copy()
df1.drop(['A','C'], axis=1)

,B
a,2.0
b,5.0
c,NaN
